In [50]:
from pynq import PL
PL.reset()
from pynq import Overlay  #import the overlay module
ol = Overlay('./design_1.bit')  #locate/point to the bit file
import pprint
#pprint.pprint(ol.ip_dict)

print(ol.ip_dict.keys())

dict_keys(['axi_dma_0', 'spi_adl_1', 'spi_lmx_0', 'spi_adl_0', 'usp_rf_data_converter_0', 'zynq_ultra_ps_e_0'])


In [4]:
#import sys

#!{sys.executable} -m pip install bitstring

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.7/71.7 KB 940.2 kB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 288.4/288.4 KB 3.9 MB/s eta 0:00:0000:01


In [51]:
import time
import bitstring

MMIO_REGISTERS = {"TRIGGER": 0x14, "SPI_DATA_IN": 0x08, "SPI_DATA_OUT": 0x0C}

class LMX2595:
    def __init__(self, mmio_spi_controller):
        self.mmio_spi_controller = mmio_spi_controller
    
    def _spi_transaction(self, command):
        self.mmio_spi_controller.write(MMIO_REGISTERS["SPI_DATA_OUT"], command)  
        self.mmio_spi_controller.write(MMIO_REGISTERS["TRIGGER"], 0x0)
        time.sleep(0.1)
        return self.mmio_spi_controller.read(MMIO_REGISTERS["SPI_DATA_IN"])
    
    def _write_register(self, address, data):
        command = bitstring.BitArray(length = 24)
        command[0] = 0 # R/W bit
        command[1:8] = address
        command[8:] = data
        return self._spi_transaction(command.u)
    
    def _read_register(self, address):
        command = bitstring.BitArray(length = 24)
        command[0] = 1 # R/W bit
        command[1:8] = address
        command[8:] = 0 # Don't care
        return self._spi_transaction(command.u)

lmx = LMX2595(ol.spi_lmx_0)

In [66]:
lmx._write_register(0, 0b0010010000011000)

16777215

In [69]:
print(bin(lmx._read_register(0)))

0b111111111111111111111111
